In [2]:
from toolbox import ToolBox
from tqdm import tqdm

from models import *

tqdm.pandas()

In [3]:
import pandas as pd

In [4]:
tb = ToolBox()

In [5]:
# Get all genres for all games
df = tb.load_data_sql(table='game_genres', use_cache=True)

Loading from database


In [6]:
### Initial DataFrame - note that one game can belong to multiple genres

,game,genre
0,007-legends-pc,Action
1,007-legends-pc,Arcade
2,007-legends-pc,First-Person
3,007-legends-pc,Modern
4,007-legends-pc,Shooter


In [17]:
### Intermediate step: list total frequency for each genre

,game,genre,freq
0,007-legends-pc,Action,7691
12718,007-legends-pc,Shooter,2432
7691,007-legends-pc,Arcade,2224
9915,007-legends-pc,First-Person,1579
11494,007-legends-pc,Modern,1224


In [16]:
### Final step: select genre with highest frequency for each game

,game,genre
0,007-legends-pc,Action
1,007-legends-playstation-3,Action
2,007-legends-wii-u,Action
3,007-legends-xbox-360,Action
4,007-quantum-of-solace-ds,Action


In [7]:
# Get the genre frequency
genre_freq = df['genre'].value_counts().to_frame()

# Rest index
genre_freq.reset_index(inplace = True)

# Rename columns
genre_freq.rename(columns={'index' : 'genre', 'genre' : 'freq'}, inplace = True)

In [8]:
genre_freq.shape, 'genres', df.shape, 'dataframe with multiple genres per game'

((169, 2), 'genres', (75484, 2), 'dataframe with multiple genres per game')

In [9]:
# Get the genre frequency
genre_freq = df['genre'].value_counts().to_frame()

# Rest index
genre_freq.reset_index(inplace = True)

# Rename columns
genre_freq.rename(columns={'index' : 'genre', 'genre' : 'freq'}, inplace = True)

# Merge initial dataframe with the frequency counts for the genres
df = pd.merge(df, genre_freq).sort_values('game')

# Find indexes for each game-genre combination with the highest frequency
idx = df.groupby(['game'])['freq'].transform(max) == df['freq']

# Create new dataframe with the assigned genre for each game
final_df = df[idx].copy()

# Drop the frequency column
final_df.drop('freq', axis = 1, inplace = True)

In [10]:
df.shape

(75484, 3)

In [12]:
# Find indexes for each game-genre combination with the highest frequency
idx = df.groupby(['game'])['freq'].transform(max) == df['freq']

In [13]:
# Create new dataframe with the assigned genre for each game
final_df = df[idx].copy()

In [14]:
# Drop the frequency column
final_df.drop('freq', axis = 1, inplace = True)

In [15]:
final_df.shape

(22378, 2)

##### Get console info

In [16]:
# Get all genres for all games
console_data = tb.load_data_sql(table='games', select='game, console', use_cache=True)

Loading from database


In [18]:
console_data.head()

,game,console
0,-arcane-preraise--pc,pc
1,-arcane-raise--pc,pc
2,-arcane-reraise--pc,pc
3,0-ji-no-kane-to-cinderella-halloween-wedding-psp,psp
4,0-kara-no-shogi-shogi-youchien-ayumi-kumi-r-pl...,playstation


#### Load dataset

In [117]:
cv_data = pd.read_csv('cross_sectional_data.csv')

In [119]:
# look at initial shape of dataframe
cv_data.shape

((22378, 2), 'genres', (39104, 2), 'consoles', (13068, 8), 'regression data')

In [121]:
# add genre column to the dataframe used to run models
cv_data = cv_data.merge(final_df, on='game', how='left')

In [133]:
# add console column to the dataframe used to run models
cv_data = cv_data.merge(console_data, on='game', how='left')

In [135]:
# confirm shape is same in lenght and increased by 2 due to added columns (genre, console)
cv_data.shape

(13068, 10)

In [139]:
# write dataframe to csv
cv_data.to_csv('cross-sectional-data-with-control-variables.csv')